This is a blog post detailing the inner workings of a baseline three-point model that can be used for sports betting in Python.

# Overview of Model

The model leverages the `nba_api` Python package and shot location data to simulate 3-point field goal attempts.

It takes into account how frequently a player shoots 3's in a game (irrespective of location), the areas on the court they most frequently shoot from (we'll call these clusters), how effective they are at shooting from these clusters, and how well their opposing matchup defends against these clusters.

It does this by simulating an amount of 3-point FGA in a game by drawing from a Poisson distribution based on a player's long run average for 3PAT/GM (we bake in some uncertainty surrounding that long-run average using boostrapping; we'll talk more about how this works in this post).  

After simulating an amount for FGA, the model then simulates <i>where</i> the player might be taking these shots by leveraging a Gaussian Mixture Model that was fitted to the shot location data in a step prior. In essence, before this step, we fit a clustering algorithm to the player's shot location data for the season (which is just X and Y data containing coordinates of where the player shot from on the court). Our fitted model allows us to sample from it as well, which is how we come up with the simulated shot locations. 

Naturally, we should expect clusters to appear in the data (Corner, wing, slot, etc.). Generally we think of 5 or so "zones" for 3-point shots. Interestingly, we let the model choose the amount of clusters via cross-validation, and for most players, it will actually choose around 5-6! Note that each player has a different set of clusters.

When we fit the data, we also measure how well a player is at shooting from these clusters, and also how well the opponents matchup defends these clusters relative to how well league defenses defends them. This gets us an adjusted-FG % per cluster.

We'll talk more about GMM in the body of the post, but essentially, it's a generalization of K-Means that allows us to estimate a probability a point belongs to the cluster. When we simulate shots around the court, we can use this matrix of probabilities to compute a weighted average for each FGA. We do some matrix operations and sum everything up at the end to get a estimated 3-point FGM per simulation. We repeat this simulation process X times and analyze the results (compare to sports betting lines).